# scite api - this works awesome!

Poppy Nicolette Riddle 2021
<br>
Creative Commons Attribution 4.0

### to do
- [ ] add in dialog to guide users
- [ ] check progress bar works on len of imported list
- [ ] data cleanup before csv export (too much repetitve work in excel)
- [ ] create a nice exportable visualization

#### references:
https://api.scite.ai/docs

In [1]:
import requests # import the requests library
import csv
import pandas as pd

## Read in a csv file
This is a one column file with all your doi's.<br>
Start with 10 or so. 500 takes about 10-15 minutes to return all the info. 

In [33]:
df = pd.read_csv ('dois.csv')
df

,doi
0,10.1007/s11192-015-1765-5
1,10.3390/su13042202
2,10.1080/09613218.2016.1086872
3,10.1016/j.habitatint.2016.06.011
4,10.1080/15623599.2017.1401290
5,10.1016/j.egypro.2016.09.106
6,10.1061/9780784481059.003
7,10.46488/NEPT.2020.V19I02.036
8,10.1016/j.clet.2020.100001
9,10.3138/9781487534905-006


In [34]:
dois = df['doi'].to_list()
print(type(dois)) #just confirming its a list and not a str
#confirming length
print('Length of list: ' + str(len(dois)))

<class 'list'>
Length of list: 19


## Send request

- [ ] what if doi's are not found?
- [x] fix progress bar - it should progress at the length of the doi list

In [35]:
from tqdm import tqdm

resp_list = []
i = 0

for i in tqdm(range(len(dois))):
    if dois[i]=='0':
        pass
    resp = requests.get('https://api.scite.ai/tallies/' + dois[i]) 
    resp_list.append(resp.text)
    i+=1
print('Done. And the length of your list is: ' + str(len(resp_list)))
print()
print("Your data format looks like: " + resp_list[0])

100%|███████████████████████████████████████████| 19/19 [00:14<00:00,  1.32it/s]

Done. And the length of your list is: 19

Your data format looks like: {"total":571,"supporting":9,"contradicting":1,"mentioning":515,"unclassified":46,"doi":"10.1007/s11192-015-1765-5","citingPublications":911}


## results - cleaning up and placed back in the dataframe
- [x] clean up results
- [x] Placed into a df so that we can visually check

In [36]:
# file cleanup
def replacer (basestr, toBeRemove, newchar) :
    for i in toBeRemove :
        if i in basestr :
            basestr = basestr.replace(i, newchar)
    return basestr

#new list
clean_list = []
i = 0 
for i in range(len(resp_list)):
    newstring = replacer(resp_list[i],["{", "}", '"', '"'], "")
    clean_list.append(newstring)
    i += 1


#add resp_list to the df
df['tallies'] = clean_list

#remove messy characters
df[['total','supporting', 'contradicting', 'mentioning', 'unclassified', 'dois', 'citing']] = df['tallies'].str.split(',',expand=True)

#drop unnecessary columns by name
df.drop('tallies', axis=1, inplace=True)
df.drop('dois', axis=1, inplace = True)

#remove words from total, supporting, etc column contents....
df['total'] = df['total'].str.replace('total:','')
df['supporting'] = df['supporting'].str.replace('supporting:','')
df['contradicting'] = df['contradicting'].str.replace('contradicting:','')
df['mentioning'] = df['mentioning'].str.replace('mentioning:','')
df['unclassified'] = df['unclassified'].str.replace('unclassified:','')
df['citing'] = df['citing'].str.replace('citingPublications:','')

df

,doi,total,supporting,contradicting,mentioning,unclassified,citing
0,10.1007/s11192-015-1765-5,571,9,1,515,46,911
1,10.3390/su13042202,0,0,0,0,0,1
2,10.1080/09613218.2016.1086872,45,1,0,44,0,93
3,10.1016/j.habitatint.2016.06.011,14,0,0,14,0,33
4,10.1080/15623599.2017.1401290,1,0,0,1,0,2
5,10.1016/j.egypro.2016.09.106,14,0,0,13,1,21
6,10.1061/9780784481059.003,4,0,0,4,0,4
7,10.46488/NEPT.2020.V19I02.036,0,0,0,0,0,0
8,10.1016/j.clet.2020.100001,2,0,0,2,0,4
9,10.3138/9781487534905-006,0,0,0,0,0,0


## write to csv file

Rename to a place and file location you like


In [30]:
#write a new csv file 
path = '/Users/nicolapoppy/Desktop/CODING_WORKING/Python/LIBRARIES/scite_ai_api'
df.to_csv (path + '/citations.csv', index = False, header=True)

print ('done')

done


## All done!
You should find the .csv file in your path location specified above. 
<br>
The original order of the imported dois file is preserved. 